# Загрузка данных / Data loading
Датасет из соревнования https://www.kaggle.com/c/nsu-abmd-2021-e-commerce#

In [ ]:
import pandas as pd



df = pd.read_csv('../input/nsu-abmd-2021-e-commerce/train.csv')
test = pd.read_csv('../input/nsu-abmd-2021-e-commerce/test.csv')
df

# Исследование данных / Data exploration
Data exploration is the first step of data analysis used to explore and visualize data to uncover insights from the start or identify areas or patterns to dig into more. Using interactive dashboards and point-and-click data exploration, users can better understand the bigger picture and get to insights faster.

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='O')

In [ ]:
df.Rating = df.Rating.str.replace(',', '.').astype(float)

In [ ]:
test.Rating = test.Rating.str.replace(',', '.').astype(float)

In [ ]:
df['Average price'].fillna((df['Max price'] + df['Min price']) / 2.0, inplace=True) 

test['Average price'].fillna((test['Max price'] + test['Min price']) / 2.0, inplace=True)

In [ ]:
df.isna().any()

In [ ]:
df = df[~pd.to_numeric(df['Days in stock'], errors='coerce').isna()]
df

In [ ]:
test = test[~pd.to_numeric(test['Days in stock'], errors='coerce').isna()]

In [ ]:
df['Days in stock'] = df['Days in stock'].astype(float)

In [ ]:
test['Days in stock'] = test['Days in stock'].astype(float)

## Количественные / Quantitative

In [ ]:
df.hist(figsize=(15, 10));

In [ ]:
df.boxplot(figsize=(20, 5), rot=45);

In [ ]:
df[df.Comments>2_000] #Price

In [ ]:
import holoviews as hv
from holoviews import dim
from holoviews import opts
hv.extension('bokeh')


opts.defaults(opts.BoxWhisker(height=120, responsive=True, toolbar='above', invert_axes=True, tools=['hover']))

def f(x):
    return hv.BoxWhisker(df[x])

hv.DynamicMap(f, kdims=['x']).redim.values(x=df.select_dtypes('number').columns)

## Категориальные / Category

In [ ]:
df.select_dtypes('O')

In [ ]:
import numpy as np


pd.set_option('min_rows', 25)
np.set_printoptions(edgeitems=100)

In [ ]:
for i in df.select_dtypes('O').columns:
    print(i, df[i].unique())

In [ ]:
df

In [ ]:
df.Deliveryscheme.value_counts(dropna=False).plot.barh();

In [ ]:
test.Deliveryscheme.value_counts(dropna=False).plot.barh();

In [ ]:
df = pd.concat([df, df.Category.str.split('/', expand=True)\
                .rename(columns={0: 'cat_meta', 1: 'cat_sub', 2: 'cat_sub_sub'})], axis=1)

In [ ]:
test = pd.concat([test, test.Category.str.split('/', expand=True)\
                .rename(columns={0: 'cat_meta', 1: 'cat_sub', 2: 'cat_sub_sub'})], axis=1)

In [ ]:
df

In [ ]:
df = pd.concat([df, df.Seller.str.split(',', expand=True)\
               .rename(columns={0: 'RealSeller', 1: 'DeliveryProvider'})], axis=1)

In [ ]:
test = pd.concat([test, test.Seller.str.split(',', expand=True)\
               .rename(columns={0: 'RealSeller', 1: 'DeliveryProvider'})], axis=1)

In [ ]:
df

In [ ]:
df.cat_meta.value_counts(dropna=False).plot.barh();
df

In [ ]:
test.cat_meta.value_counts(dropna=False).plot.barh();

In [ ]:
df.boxplot(column='Sales', by='cat_meta', figsize=(5, 10), vert=False);

In [ ]:
df.cat_sub.value_counts(dropna=False)

In [ ]:
df

In [ ]:
df.cat_sub.value_counts(dropna=False, ascending=True).plot.barh(figsize=(5, 50), );

In [ ]:
df

In [ ]:
df.loc[df.groupby('cat_sub').Sales.transform('count')<1000, 'cat_sub'] = 'Rare'

In [ ]:
df.groupby(['cat_sub'])['Sales'].max().sort_values(ascending=True)

In [ ]:
df

In [ ]:
t = df.groupby(['cat_sub'])['Sales'].median().sort_values(ascending=True)
df['cat_sub'] = df['cat_sub'].astype(pd.CategoricalDtype(list(t.index), ordered=True))
df.boxplot(column='Sales', by='cat_sub', figsize=(10, 25), vert=False);

In [ ]:
test['cat_sub'] = test['cat_sub'].astype(pd.CategoricalDtype(list(t.index), ordered=True))

In [ ]:
test['cat_sub'] = test['cat_sub'].astype(pd.CategoricalDtype(list(t.index), ordered=True))


In [ ]:
df

In [ ]:
df.cat_sub_sub.value_counts(dropna=False)

In [ ]:
df[df.groupby('cat_sub_sub').Sales.transform('count')>1].Sales.value_counts(bins=25)#.cat_sub_sub.value_counts(dropna=False)

In [ ]:
df[df.groupby('cat_sub_sub').Sales.transform('count')>1]

In [ ]:
df[df.groupby('cat_sub_sub').Sales.transform('count')>500]\
    .boxplot(column='Sales', by='cat_sub_sub', figsize=(10, 150), vert=False);

In [ ]:
# df.loc[df.groupby('cat_sub_sub').Sales.transform('count')>1000, 'cat_sub'] = 'Rare'

In [ ]:
df.Brand.value_counts(dropna=False)

In [ ]:
test.Brand.value_counts(dropna=False)

In [ ]:
df[df.groupby('Brand').Sales.transform('count')>1].Sales.value_counts(bins=15)#.cat_sub_sub.value_counts(dropna=False)

In [ ]:
df.loc[df.groupby('Brand').Sales.transform('count')<300, 'Brand'] = 'Rare'

In [ ]:
t = df.groupby(['Brand'])['Sales'].median().sort_values()
df['Brand'] = df['Brand'].astype(pd.CategoricalDtype(list(t.index), ordered=True))

df.boxplot(column='Sales', by='Brand', figsize=(10, 100), vert=False);

In [ ]:
test['Brand'] = test['Brand'].astype(pd.CategoricalDtype(list(t.index), ordered=True))

In [ ]:
df

# Чистка данных / Data cleaning
The main aim of Data Cleaning is to identify and remove errors & duplicate data, in order to create a reliable dataset. This improves the quality of the training data for analytics and enables accurate decision-making.

## Пустые значения / Missing values

In [ ]:
# for i in df.select_dtypes('number').columns:
#     print(i, df[i].unique())

In [ ]:
df.info()

In [ ]:
test.info()

In [ ]:
df_nan = df[['Price', 'Brand', 'cat_meta', 'cat_sub', 'cat_sub_sub', 'RealSeller', 'Max price', 'Min price', 'Average price', 
             'Deliveryscheme', 'Comments', 'Days in stock', 'Rating', 'Sales']].copy()

df_nan.info()

In [ ]:
test_nan = test[['Price', 'Brand', 'cat_meta', 'cat_sub', 'Max price', 'Min price', 'Average price', 'cat_sub_sub', 'RealSeller', 
             'Deliveryscheme', 'Comments', 'Days in stock', 'Rating']].copy()

test_nan.info()

In [ ]:
df_nan.isna().any()

In [ ]:
df_nan[df_nan.isna().any(axis=1)]#.style.highlight_null()

In [ ]:
df_nan['Average price'].fillna((df_nan['Max price']+df_nan['Min price']) / 2, inplace=True)

In [ ]:
df_nan.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
df_nan.info()

In [ ]:
test_nan.info()

In [ ]:
df_nan

## One-hoting

In [ ]:
test_2_nan = test_nan.copy()
test_2_nan.info()

In [ ]:
df_one_hoted = df_nan.copy()

ONE_HOT_COL = ['Deliveryscheme'] #, 'Brand', 'cat_meta', 'cat_sub'

temp = pd.get_dummies(df_one_hoted[ONE_HOT_COL], prefix=ONE_HOT_COL, dtype=bool)

df_one_hoted = pd.concat([temp, df_one_hoted.select_dtypes('number'), 
                            df_one_hoted.select_dtypes('bool')], axis=1)

df_nan.shape, df_one_hoted.shape

In [ ]:
df_nan.info()

In [ ]:
test_one_hoted = test_2_nan.copy()

ONE_HOT_COL = ['Deliveryscheme'] #, 'Brand', 'cat_meta', 'cat_sub'

temp = pd.get_dummies(test_one_hoted[ONE_HOT_COL], prefix=ONE_HOT_COL, dtype=bool)

test_one_hoted = pd.concat([temp, test_one_hoted.select_dtypes('number'), 
                            test_one_hoted.select_dtypes('bool')], axis=1)

test_nan.shape, test_one_hoted.shape

In [ ]:
test_one_hoted.info()

In [ ]:
df_one_hoted.info()

In [ ]:
df_one_hoted.isna().any()

# Создание модели / Model creation

In [ ]:
from sklearn.model_selection import train_test_split

df_sampled = df_one_hoted.sample(700_000)

X_train, X_test, y_train, y_test = train_test_split(df_sampled.drop(columns='Sales'), df_sampled.Sales)

In [ ]:
test_ = test_one_hoted.copy()


In [ ]:
test_.info()

In [ ]:
df_sampled.info()

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor


model = RandomForestRegressor(n_jobs = -1, verbose = 2, max_depth=18, random_state=0)
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
y_pred = model.predict(test_)


In [ ]:
temp = pd.DataFrame(y_pred, columns=['Expected'])
temp['Expected'] = temp['Expected'].astype(int)
temp['Id'] = temp.index
temp = temp[['Id', 'Expected']]
temp.head()
temp.Expected = temp.Expected.apply(lambda x: 0 if x < 0 else x)
temp.head()
temp.to_csv('./result.csv',index = False)
temp.head()